In [1]:
import re
import numpy as np

import itertools

https://adventofcode.com/2020

# Day 1 

In [ ]:
input1 = np.loadtxt('input1', dtype=int)

## Part 1

In [13]:
matches = []
for vals in itertools.combinations(input1, 2):
    if np.sum(vals) == 2020:
        matches.append(vals)
        
assert len(matches) == 1
np.prod(matches[0])

471019

## Part 2 

In [14]:
matches = []
for vals in itertools.combinations(input1, 3):
    if np.sum(vals) == 2020:
        matches.append(vals)
        
assert len(matches) == 1
np.prod(matches[0])

103927824

# Day 2

In [17]:
policies = [] # letter, (mn, mx)
pws = []

with open('input2') as f:
    for line in f:
        policy, pw = line.split(': ')
        pws.append(pw.strip())
        
        rng, letter = policy.split()
        rngs = [int(e) for e in rng.split('-')]
        policies.append((letter, tuple(rngs)))

('t', (3, 4))

## Part 1

In [24]:
valid = []
for (letter, (mn, mx)), pw in zip(policies, pws):
    nletter = pw.count(letter)
    valid.append(nletter >= mn and nletter <= mx)
np.sum(valid)

586

## Part 2 

In [27]:
valid = []
for (letter, (mn, mx)), pw in zip(policies, pws):
    match1 = pw[mn-1] == letter
    match2 = pw[mx-1] == letter
    valid.append(match1 != match2) # effectively XOR
np.sum(valid)

352

# Day 3

In [27]:
trees = []
with open('input3') as f:
    for line in f:
        trees.append(np.array(list(line.strip()))=='#')
trees = np.array(trees)
trees.shape

(323, 31)

## Part 1

In [25]:
rightperdown = 3

In [30]:
ntrees = 0
j = 0
for i in range(trees.shape[0]):
    ntrees += trees[i, j%trees.shape[1]]
    j += rightperdown
ntrees

242

In [31]:
ntrees_part1 = ntrees

## Part 2 

In [32]:
scenarios_to_check = [(1, 1),
                      (3, 1),
                      (5, 1),
                      (7, 1),
                      (1, 2)]

In [34]:
def check_trees(right, down):
    ntrees = 0
    i = j = 0
    while i < trees.shape[0]:
        ntrees += trees[i, j%trees.shape[1]]
        i += down
        j += right
    return ntrees
assert check_trees(rightperdown, 1) == ntrees_part1

In [36]:
ntrees_per_scenario = [check_trees(*scen) for scen in scenarios_to_check]
np.prod(ntrees_per_scenario)

2265549792

# Day 4

In [75]:
fields = {'byr':'Birth Year',
            'iyr':'Issue Year',
            'eyr':'Expiration Year',
            'hgt':'Height',
            'hcl':'Hair Color',
            'ecl':'Eye Color',
            'pid':'Passport ID',
            'cid':'Country ID'}

In [76]:
with open('input4') as f:
    iddocs = f.read().strip().split('\n\n')
iddocs = [doc.replace('\n', ' ') for doc in iddocs]
iddocs = [dict([kv.split(':') for kv in doc.split(' ')]) for doc in iddocs]
iddocs[-1]

{'cid': '244',
 'hcl': '#866857',
 'ecl': 'amb',
 'byr': '1931',
 'eyr': '1928',
 'pid': '557376401',
 'hgt': '182cm',
 'iyr': '2013'}

## Part 1

In [77]:
required_fields = set(fields.keys())
required_fields.remove('cid')
optional_fields = {'cid'}
all_valid_fields = optional_fields.union(required_fields)

valids = []

for doc in iddocs:
    doc_fields = set(doc.keys())
    
    # check if all required fields are present
    if required_fields.difference(doc_fields):
        valids.append(False)
    # check if there's an invalid field
    elif doc_fields.difference(all_valid_fields):
        valids.append(False)
    else:
        valids.append(True)
        
sum(valids)

196

## Part 2 

In [78]:
required_fields = set(fields.keys())
required_fields.remove('cid')
optional_fields = {'cid'}
all_valid_fields = optional_fields.union(required_fields)

valids = []
invalid_docs = []
valid_docs = []

for doc in iddocs:
    doc_fields = set(doc.keys())
    
    # check if all required fields are present
    if required_fields.difference(doc_fields):
        valids.append(False)
    # check if there's an invalid field
    elif doc_fields.difference(all_valid_fields):
        valids.append(False)
    else:
        # valid fields, now check the values
        if not (1920 <= int(doc['byr']) <= 2002):
            valids.append(False)
        elif not (2010 <= int(doc['iyr']) <= 2020):
            valids.append(False)
        elif not (2020 <= int(doc['eyr']) <= 2030):
            valids.append(False)
        elif not re.match('#[0-9a-f]{6}', doc['hcl']):
            valids.append(False)
        elif doc['ecl'] not in ('amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'):
            valids.append(False)
        elif not re.match('[0-9]{9}', doc['pid']):
            valids.append(False)
        else:
            # treat height separately due to the complex unit parsing
            match = re.match('(\d*)(cm|in)', doc['hgt'])
            if match:
                num, unit = match.groups()
                if unit == 'cm':
                    valids.append(150 <= int(num) <=193)
                elif unit == 'in':
                    valids.append(59 <= int(num) <=76)
                else:
                    assert False, 'should be impossible'
            else:
                valids.append(False)
        if valids[-1]:
            valid_docs.append(doc)
        else:
            invalid_docs.append(doc)
        
sum(valids)

115

# Day 5

In [13]:
with open('input5') as f:
    boarding_passes = f.read().strip().split('\n')

## Part 1

In [10]:
def bsp_seat(inpt):
    rowstr = inpt[:7]
    
    scale = 2**len(rowstr)
    row_num = 0
    for e in rowstr:
        scale //= 2
        if e == 'B':
            row_num += scale
        elif e == 'F':
            pass
        else:
            raise ValueError('invalid value encountered in finding row')
    assert scale == 1  # row_num is now the correct row
    
    seatstr = inpt[7:]
    scale = 2**len(seatstr)
    seat_num = 0
    for e in seatstr:
        scale //= 2
        if e == 'R':
            seat_num += scale
        elif e == 'L':
            pass
        else:
            raise ValueError('invalid value encountered in finding seat')
        
    return row_num, seat_num
        
    
test_input0 = 'FBFBBFFRLR'
row, seat = bsp_seat(test_input0)
assert row == 44
assert seat == 5

In [16]:
def rowcol_to_id(row, seat):
    return row*8 + seat

assert rowcol_to_id(*bsp_seat('BFFFBBFRRR')) == 567
assert rowcol_to_id(*bsp_seat('FFFBBBFRRR')) == 119
assert rowcol_to_id(*bsp_seat('BBFFBBFRLL')) == 820

In [18]:
ids = [rowcol_to_id(*bsp_seat(bpstr)) for bpstr in boarding_passes]
max(ids)

926

## Part 2 

In [35]:
sortids = np.sort(ids)

In [39]:
idx_skip = np.where(np.diff(sortids)==2)[0][0]
sortids[idx_skip:idx_skip+2]

array([656, 658])

In [40]:
my_seat = sortids[idx_skip]+1
my_seat

657

# Day 6

In [111]:
with open('input6') as f:
    input6 = f.read().strip()

In [30]:
qnames = tuple([chr(97+i) for i in range(26)])

## Part 1

In [5]:
test_input = """abc

a
b
c

ab
ac

a
a
a
a

b"""

def parse_groups(inpt):
    groups = []
    for group in inpt.split('\n\n'):
        groups.append([tuple(answers) for answers in group.split('\n')])
    return groups
    
parse_groups(test_input)

[[('a', 'b', 'c')],
 [('a',), ('b',), ('c',)],
 [('a', 'b'), ('a', 'c')],
 [('a',), ('a',), ('a',), ('a',)],
 [('b',)]]

In [112]:
def count_any_yeses(groups):
    group_yeses = []
    for group in groups:
        qs_yesed = set()
        for person in group:
            for q in person:
                qs_yesed.add(q)
        group_yeses.append(len(qs_yesed))
    return group_yeses

answers = count_any_yeses(parse_groups(test_input))
assert np.all(np.array([3, 3, 3, 1, 1])==answers)
sum(answers)

11

In [113]:
sum(count_any_yeses(parse_groups(input6)))

6714

## Part 2 

In [114]:
def count_all_yeses(groups):
    group_yeses = []
    for group in groups:
        qs_yesed = set(qnames)
        for person in group:
            persons_yeses = set(person)
            qs_yesed = persons_yeses.intersection(qs_yesed)
        group_yeses.append(len(qs_yesed))
    return group_yeses

answers = count_all_yeses(parse_groups(test_input))
assert np.all(np.array([3, 0, 1, 1, 1])==answers)
sum(answers)

6

In [115]:
sum(count_all_yeses(parse_groups(input6)))

3435

# Day 7

In [2]:
with open('input7') as f:
    input7 = f.read().strip()

## Part 1

In [3]:
test_input = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

In [69]:
def parse_lines(inpt):
    if isinstance(inpt, str):
        inpt = inpt.split('\n')
        
    bag_contents = {}
    for line in inpt:
        bag, contains = re.match('(.*) bags contain (.*)', line).groups()
        if contains == 'no other bags.':
            bag_contents[bag] = tuple()
        else:
            contents = []
            for innerstr in contains.split(', '):
                num, innerbag = re.match('([0-9]+) (.*) bag', innerstr).groups()
                contents.append((int(num), innerbag))
                
            bag_contents[bag] = tuple(contents)
        
    return bag_contents

test_parsed = parse_lines(test_input)
test_parsed

{'light red': ((1, 'bright white'), (2, 'muted yellow')),
 'dark orange': ((3, 'bright white'), (4, 'muted yellow')),
 'bright white': ((1, 'shiny gold'),),
 'muted yellow': ((2, 'shiny gold'), (9, 'faded blue')),
 'shiny gold': ((1, 'dark olive'), (2, 'vibrant plum')),
 'dark olive': ((3, 'faded blue'), (4, 'dotted black')),
 'vibrant plum': ((5, 'faded blue'), (6, 'dotted black')),
 'faded blue': (),
 'dotted black': ()}

In [70]:
def recurse_contain(container, target, dct):
    if dct[container]: # not empty
        for num, inner_container in dct[container]:
            if inner_container == target:
                return True
            elif recurse_contain(inner_container, target, dct):
                return True
            # otherwise keep going
            
    return False
    
matches = [recurse_contain(name, 'shiny gold', test_parsed) for name in test_parsed.keys()]
assert sum(matches) == 4
assert sum(matches[:4]) == 4  # only works in py versions where order is preserved in dicts

Real thign

In [71]:
parsed7 = parse_lines(input7)
sum([recurse_contain(name, 'shiny gold', parsed7) for name in parsed7.keys()])

112

## Part 2 

In [72]:
test_input2 = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags."""

test_parsed2 = parse_lines(test_input2)
test_parsed2

{'shiny gold': ((2, 'dark red'),),
 'dark red': ((2, 'dark orange'),),
 'dark orange': ((2, 'dark yellow'),),
 'dark yellow': ((2, 'dark green'),),
 'dark green': ((2, 'dark blue'),),
 'dark blue': ((2, 'dark violet'),),
 'dark violet': ()}

In [79]:
def recurse_count(target, dct):
    count = 0
    for n, bagtype in dct[target]:
        ninsideper = recurse_count(bagtype, dct)
        count += n * (ninsideper + 1)
    return count

assert recurse_count('shiny gold', test_parsed2) == 126

In [80]:
recurse_count('shiny gold', parsed7)

6260

# Day N

## Part 1

## Part 2 